# PROJECT 2 
(Natural Language Processing)
### AUTHORS
[UJJWAL KUMAR : 260730680]

[ABDULLAHI ELMI : 260727124]

[FURAHA DAMIEN : 260754407]

### IMPORTS

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import os
from sklearn import datasets
import sys

In [ ]:

classes = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware',
           'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos',
           'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'talk.politics.misc',
           'talk.politics.guns', 'talk.politics.mideast', 'sci.crypt', 'sci.electronics',
           'sci.med', 'sci.space', 'talk.religion.misc', 'alt.atheism', 'soc.religion.christian']


## LOADING DATASETS

In [ ]:

# twenty_test = None
# twenty_train = None
# docs_test = None
# '''
# NOT FOR .ipynb files
#  Ask user for which data set they wanna test'''
# dataset_name =sys.argv[1]
# print("You choose dataset : ", sys.argv)
# if(dataset_name == "imdb"):
#     #  Training set for IMDB datasets
#     twenty_train = datasets.load_files("../dataset/data/train", description=None, categories=None,
#                                        load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)
#     #  IMDB test dataset
#     twenty_test = datasets.load_files("../dataset/data/test", description=None, categories=None,
#                                       load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)
#     docs_test = twenty_test.data
# else:
#     # Training set for 20 news groups dataset
#     twenty_train = fetch_20newsgroups(
#         subset='train', categories=classes, shuffle=True, random_state=42)
#     # TEST SET for 20 new group
#     twenty_test = fetch_20newsgroups(
#         subset='test', categories=classes, shuffle=True, random_state=42)
#     docs_test = twenty_test.data


# load data
path = os.getcwd()
print(".... Loading training DATASET..... ")
print(".....Done......")
''' Training set for IMDB datasets'''
twenty_train = datasets.load_files("../dataset/data/train", description=None, categories=None,
                                   load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)

''' Training set for 20 news groups dataset '''
# twenty_train = fetch_20newsgroups(
#     subset='train', categories=classes, shuffle=True, random_state=42)




## TEXT PROCESSING PIPELINE

In [ ]:
"""
PRE-PROCESSING
    constructing bag of words
    tokenizing and filtering of stopwords
    feature extraction
    counts of N-grams of words
    downscale word weights by frequency
"""
# creating pipeline


class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()

    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]


print("Creating text processing pipline... ")

def PIPELINE(classifier):
    pipeline_1 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', classifier)
                           ])
    pipeline_2 = Pipeline([('vect', CountVectorizer(tokenizer=LemmaTokenizer(),
                                                    strip_accents='unicode',  # works
                                                    stop_words='english',  # works
                                                    lowercase=True,)), ('tfidf', TfidfTransformer()), ('clf', classifier)
                           ])
    return pipeline_1


count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)


print(".... Loading testing DATASET..... ")
print(".....Done......")
# TEST SET
# twenty_test = fetch_20newsgroups(subset='test',
#                                  categories=classes, shuffle=True, random_state=42)

''' IMDB test dataset'''
twenty_test = datasets.load_files("../dataset/data/test", description=None, categories=None,
                                  load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)
docs_test = twenty_test.data



# LOGISTIC REGRESSION


In [ ]:
# TRAIN CLASSIFIERS
# =====================LOGISTIC REGRESSION========================
print("================== LOGISTIC REGRESSION =======================")
classifier = LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
                                intercept_scaling=1, l1_ratio=None, max_iter=100,
                                multi_class='auto', n_jobs=None, penalty='l2',
                                random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                                warm_start=False)
# pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', classifier)
#                      ])
pipeline = PIPELINE(classifier)

pipeline.fit(twenty_train.data, twenty_train.target)
predicted = pipeline.predict(docs_test)

''' Classification for 20 news group dataset'''
# print(metrics.classification_report(twenty_test.target, predicted,
#                                     target_names=twenty_test.target_names))
'''Classification report for IMDB dataset '''
print(metrics.classification_report(twenty_test.target, predicted))

# print(metrics.confusion_matrix(twenty_test.target, predicted))
print("Raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))




## GRID SEARCH
(LOGISTIC REGRESSION)

In [ ]:
# Logistic Regression parameter tuning with GridSearchCV
# TODO: Add more hyperparameter tuning
# pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())
#                      ])
# regularization hyperparameter space

hyperparameters = {
    'clf__C': [0.1, 1, 100, 1000],
    # 'clf__penalty': ['l1', 'l2'],
    'clf__max_iter': [7600, 1000],
    'clf__tol': [0.00001, 0.0001, 0.005]
}
grid_logistic = GridSearchCV(pipeline, hyperparameters, cv=5, )

clf_best = grid_logistic.fit(twenty_train.data, twenty_train.target)
print("<< REPORT >>")
print("Best score accurracy = %.3f%%" % ((clf_best.best_score_)*100.0))
print("Best parameters are:")
print(clf_best.best_params_)

# DECISION TREES

In [ ]:
# ===================== DECISION TREES ========================
print("================== DECISION TREES =======================")
clf_decision_trees = DecisionTreeClassifier(random_state=0)
# pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', clf_decision_trees)
#  ])
pipeline = PIPELINE(clf_decision_trees)
pipeline.fit(twenty_train.data, twenty_train.target)
predicted = pipeline.predict(docs_test)


''' Classification for 20 news group dataset'''
# print(metrics.classification_report(twenty_test.target, predicted,
#                                     target_names=twenty_test.target_names))
'''Classification report for IMDB dataset '''
print(metrics.classification_report(twenty_test.target, predicted))
# print(metrics.confusion_matrix(twenty_test.target, predicted))


print("raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))
print(cross_val_score(pipeline, twenty_train.data, twenty_train.target, cv=10))



## GRID SEARCH
(DECISION TREES)

In [ ]:
# Decision trees parameter tuning with GridSearchCV
# pipeline = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', DecisionTreeClassifier())
#                      ])
hyperparameters = {
    'clf__criterion': ['gini', 'entropy'],
    'clf__splitter': ['best', 'random'],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__class_weight': ['balanced']
}
grid_d_trees = GridSearchCV(pipeline, hyperparameters, cv=5)
clf_best = grid_d_trees.fit(
    twenty_train.data, twenty_train.target)
print("<< REPORT >>")
print("Best score accurracy = %.3f%%" % ((clf_best.best_score_)*100.0))
print("Best parameters are:")
print(clf_best.best_params_)

# SUPPORT VECTOR MACHINE


In [ ]:
# =====================SVM - LINEARSVC ========================
print("================== SUPPORT VECTOR MACHINE =======================")
classifier = LinearSVC(loss='hinge', penalty='l2',
                       random_state=0, max_iter=76000, tol=1e-5)
# clf = Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', LinearSVC(loss='hinge', penalty='l2',
#                       random_state=0, max_iter=76000, tol=1e-5)),
# ])
clf = PIPELINE(classifier)
clf.fit(twenty_train.data, twenty_train.target)
predicted = clf.predict(docs_test)


''' Classification for 20 news group dataset'''
# print(metrics.classification_report(twenty_test.target, predicted,
#                                     target_names=twenty_test.target_names))
'''Classfication report for IMDB dataset  '''
print(metrics.classification_report(twenty_test.target, predicted))

# print(metrics.confusion_matrix(twenty_test.target, predicted))


print("raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))



## GRID SEARCH
(SUPPORT VECTOR MACHINE)

In [ ]:
# LinearSVC parameter tuning with GridSearchCV
# clf=Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', LinearSVC())
# ])

parameters = {
    'tfidf__use_idf': (True, False),
    'clf__tol': [1e-2, 1e-4, 1e-9],
    'clf__max_iter': [7600, 1000]
}

gs_clf = GridSearchCV(clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

print("<< REPORT >>")
print("Best score accurracy = %.3f%%" % ((gs_clf.best_score_)*100.0))
print("Best parameters are : ")
print(gs_clf.best_params_)

# ADABOOST CLASSIFIER


In [ ]:

# =====================ADABOOST CLASSIFIER========================
print("================== ADABOOST CLASSIFIER =======================")
classifier = AdaBoostClassifier(n_estimators=100, random_state=0)
# clf_adaboost = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', AdaBoostClassifier(n_estimators=100, random_state=0))
#                          ])
clf_adaboost = PIPELINE(classifier)
clf_adaboost.fit(twenty_train.data, twenty_train.target)
predicted = clf_adaboost.predict(docs_test)


''' Classification for 20 news group dataset'''
# print(metrics.classification_report(twenty_test.target, predicted,
#                                     target_names=twenty_test.target_names))

'''Classification for MDB dataset '''
print(metrics.classification_report(twenty_test.target, predicted))

# print(metrics.confusion_matrix(twenty_test.target, predicted))


print("raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))





## GRID SEARCH
(ADABOOST CLASSIFIER)

In [ ]:
# AdaBoostClassifier parameter tuning with GridSearchCV
# clf = Pipeline([
#     ('vect', CountVectorizer()),
#     ('tfidf', TfidfTransformer()),
#     ('clf', AdaBoostClassifier())
# ])

parameters = {
    'clf__n_estimators': [20, 50, 70, 100],
    'clf__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
}

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
gs_clf = GridSearchCV(clf_adaboost, parameters, cv=kfold, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

print("<< REPORT >>")
print("Best score accurracy = %.3f%%" % ((gs_clf.best_score_)*100.0))
print("Best parameters are : ")
print(gs_clf.best_params_)

# RANDOM FOREST


In [ ]:

# =====================RANDOM FOREST========================
print("================== RANDOM FOREST =======================")

classifier = RandomForestClassifier(max_depth=2, random_state=0)
# clf_random_forest = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', RandomForestClassifier(max_depth=2, random_state=0))
# ])
clf_random_forest = PIPELINE(classifier)
clf_random_forest.fit(twenty_train.data, twenty_train.target)
predicted = clf_random_forest.predict(docs_test)


# ======= meteric classification
''' Classification for 20 news group dataset'''
# print(metrics.classification_report(twenty_test.target, predicted,
#                                     target_names=twenty_test.target_names))

'''Classification for IMDB dataset '''
print(metrics.classification_report(twenty_test.target, predicted))
# print(metrics.confusion_matrix(twenty_test.target, predicted))


print("raw accurracy = %.3f%%" %
      ((np.mean(predicted == twenty_test.target))*100.0))



## GRID SEARCH
(RANDOM FOREST)

In [ ]:
# AdaBoostClassifier parameter tuning with GridSearchCV
# clf_random_forest=Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', RandomForestClassifier())
#                               ])
param_grid = {
    'clf__n_estimators': [200, 500],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__max_depth': [4, 5, 6, 7, 8],
    'clf__criterion': ['gini', 'entropy']
}
s
grid_random_forest = GridSearchCV(
    clf_random_forest, param_grid, cv=5, n_jobs=-1)
grid_random_forest.fit(twenty_train.data, twenty_train.target)
predicted = grid_random_forest.predict(docs_test)

print("<< REPORT >>")
print("Best score accurracy = %.3f%%" %
      ((grid_random_forest.best_score_)*100.0))
print("best parameters are: ")
print(grid_random_forest.best_params_)
